## Priprema ParlaSpeechHR .json filea za obradu:

* DIPL.FON: frekvencija glasnika
* JKIM: spomen mladih, studenata, djece
* STAT: seminar neš idk

In [ ]:
#pip install jsonlines

In [7]:
import pandas as pd
import pickle
import jsonlines
import unicodedata

In [9]:
# # Decoding unicode to Latin
# def normalize_words(words_list):
#     return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8') for word in words_list]


# # Read the JSON Lines file
# data = []
# with jsonlines.open('ParlaSpeech-HR.v1.0.jsonl', 'r') as file:
#     for line in file:
#         # Normalize specific fields to ASCII (change 'field_to_normalize' accordingly)
#         line['words'] = normalize_words(line['words'])
#         data.append(line)
        
# print("JSONL normalized ASCII and loaded!")

# print(f"Sorting data by: utterance_id_start")
# # Sort the data by a specific field (e.g., 'utterance_id_start')
# sorted_data = sorted(data, key=lambda x: (x.get('orig_file'), x.get('utterance_id_start'), x.get('start')))
# print(f"Sorted data by: utterance_id_start")

# # Convert the sorted data to a DataFrame (optional but useful)
# df = pd.DataFrame(sorted_data)

# # Save as CSV
# df.to_csv('sorted_ParlaSpeech-HR.v1.0.csv', index=False)
# print("saved as CSV")


JSONL normalized ASCII and loaded!
Sorting data by: utterance_id_start
Sorted data by: utterance_id_start
saved as CSV


In [15]:
# Mapping dictionary for specific character replacements
unicode_mapping = {
    '\u0161': 'š',  # š
    '\u017e': 'ž',  # ž
    '\u010d': 'č',  # č
    '\u0111': 'đ',  # đ
    '\u0107': 'ć'   # ć
}

# Function to normalize words using the mapping dictionary
def normalize_words(words_list):
    normalized_words = []
    for word in words_list:
        normalized_word = ''.join(unicode_mapping.get(char, char) for char in word)
        normalized_words.append(normalized_word)
    return normalized_words

In [ ]:
# # Read the JSON Lines file
# print("Loading JSONL file")
# data = []
# with jsonlines.open('ParlaSpeech-HR.v1.0.jsonl', 'r') as file:
#     for line in file:
#         # Normalize the list of words in the 'words' field using the mapping dictionary
#         line['words'] = normalize_words(line['words'])
#         data.append(line)

# print("Sorting by orig_file, utterance_id_start and start...")
# # Sort the data by a specific field (e.g., 'utterance_id_start')
# sorted_data = sorted(data, key=lambda x: (x.get('orig_file'), x.get('utterance_id_start'), x.get('start')))

# # Convert the sorted data to a DataFrame (optional but useful)
# df = pd.DataFrame(sorted_data)


# columns_to_drop = ['word_start_times',
#                    'utterance_id_end',
#                    'norm_words_start_times',
#                    'norm_words',
#                    'end',
#                    'split',
#                    'norm_words_edited',
#                    'path',
#                    'utterance_id_start']
# print(f"Dropping columns: {columns_to_drop}")

# # Drop specified columns from the DataFrame
# df.drop(columns=columns_to_drop, inplace=True)


# # Save as CSV
# df.to_csv('sorted_ParlaSpeech-HR.v1.1.csv', index=False)
# print("Saved as .CSV!")

In [34]:
# Read the JSON Lines file

import jsonlines
import pandas as pd
import codecs

# Function to replace specific Unicode characters in words_list
def replace_unicode_chars(words_list):
    replacements = {
        '\u0161': 'š',  # š
        '\u017e': 'ž',  # ž
        '\u010d': 'č',  # č
        '\u0111': 'đ',  # đ
        '\u0107': 'ć'   # ć
    }
    return ["".join(replacements.get(char, char) for char in word) for word in words_list]
    
print("Loading JSONL file")
# Read the JSON Lines file and process the data
data = []
with codecs.open('ParlaSpeech-HR.v1.0.jsonl', 'r', encoding='utf-8') as file:
    reader = jsonlines.Reader(file)
    for line in reader:
        line['words'] = replace_unicode_chars(line['words'])
        data.append(line)

print("Sorting by orig_file, utterance_id_start and start...")
# Sort the data by a specific field (e.g., 'utterance_id_start')
sorted_data = sorted(data, key=lambda x: (x.get('orig_file'), x.get('utterance_id_start'), x.get('start')))

# Convert the sorted data to a DataFrame (optional but useful)
df = pd.DataFrame(sorted_data)


columns_to_drop = ['word_start_times',
                   'utterance_id_end',
                   'norm_words_start_times',
                   'norm_words',
                   'end',
                   'split',
                   'norm_words_edited',
                  # 'path',
                  # 'utterance_id_start'
                  ]
print(f"Dropping columns: {columns_to_drop}")

# Drop specified columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)


# Save as CSV
df.to_csv('sorted_ParlaSpeech-HR.v1.1.csv', index=False)
print("Saved as .CSV!")

Loading JSONL file
Sorting by orig_file, utterance_id_start and start...
Dropping columns: ['word_start_times', 'utterance_id_end', 'norm_words_start_times', 'norm_words', 'end', 'split', 'norm_words_edited']
Saved as .CSV!


In [35]:
df_copy = df.copy()

# Join the lists of words into a single sentence in a new column 'sentence'
df_copy['sentence'] = df_copy['words'].apply(lambda x: ' '.join(x)).str.lower()

# Drop the original 'words' column if not needed anymore
df_copy.drop(columns='words', inplace=True)

print(df_copy['sentence'])

0         konačni prijedlog zakona o izmjenama i dopunam...
1         u svezi sa člankom 190. poslovnika hrvatskog s...
2         za zakonodavstvo i odbor za financije i državn...
3         hvala lijepo gospodine potpredsjedniče hrvatsk...
4         zakona o izmjenama i dopunama zakona o doprino...
                                ...                        
403920    hrvatski narod direktno odlučuje, a kad se rad...
403921    naše domovine andrej plenković koji je neuspje...
403922                  zanima me hoće li andrej plenković,
403923    znači po pitanju novog zakona o referendumima ...
403924    pa ćemo mi se često ugledati na zemlje zapadne...
Name: sentence, Length: 403925, dtype: object


In [39]:
# Group by 'orig_file' and aggregate 'sentence' into a single text chunk
df_combined = df_copy.groupby('path')['sentence'].apply(' '.join).reset_index()
print(df_combined['sentence'])

0         lijepo. kolegice i kolege, nemojte se vi varda...
1         hdz želi pokušati dati obećanje za 6 mjeseci d...
2         ste imali rast 3%, a sada predlaže kao fol, g....
3         ma dajte molim vas, to nitko ne vjeruje, to ne...
4         djeca mogu stjecati znanje i obrazovati se. da...
                                ...                        
403920    prosječna plaća u hrvatskoj, svijež podatak ne...
403921    krenuti ću na onom dijelu gdje se opisuju poje...
403922    poznavanje niza propisa da bi uopće mogao polo...
403923    pa konkretno i nekih drugih stvari, drugih zak...
403924    tako da prije svega, moramo, htio bi na taj na...
Name: sentence, Length: 403925, dtype: object


In [38]:
# Save as CSV
df_combined.to_csv('blokovi_tekstova_ParlaSpeech-HR.v1.1.csv', index=False)
print("Saved as .CSV!")

Saved as .CSV!


In [40]:
from collections import Counter

# Concatenate all sentences into a single string
all_text = ' '.join(df_combined['sentence'])

# Count the frequency of each character
character_frequency = Counter(all_text.lower())  # Convert to lowercase for uniform counting

# Display the character frequencies
print(character_frequency)


Counter({' ': 14419033, 'a': 8473320, 'o': 7134726, 'i': 7050533, 'e': 6080196, 'n': 4648748, 't': 3651668, 'r': 3511665, 'j': 3419250, 's': 3255595, 'd': 2834497, 'u': 2828080, 'v': 2702877, 'm': 2543126, 'k': 2461565, 'l': 2239951, 'p': 2027631, 'z': 1281364, 'g': 1117595, ',': 966159, 'b': 952252, 'š': 681802, '.': 633399, 'č': 565108, 'h': 519356, 'c': 512317, 'ć': 477359, 'ž': 410163, 'đ': 163320, 'f': 122684, '0': 121397, '1': 73991, '2': 69318, '-': 60284, '5': 36839, '?': 34026, '3': 32637, '4': 23869, '6': 21275, '7': 20024, '8': 18816, '%': 18226, '9': 17383, "'": 4564, '_': 3838, '„': 3129, '“': 3083, 'x': 2429, '/': 1820, ':': 1656, 'w': 1581, 'y': 1202, '!': 416, ')': 352, '(': 258, '+': 238, 'q': 134, ';': 77, '"': 41, 'ü': 39, '¾': 35, '&': 26, '–': 22, '¼': 22, '>': 10, '—': 10, '@': 10, '’': 8, 'ö': 7, '̊': 6, '=': 4, '#': 4, '¸': 3, 'ӧ': 3, '‰': 3, '*': 2, '€': 2, 'ä': 2, '°': 1, '½': 1, '´': 1, '′': 1, 'ô': 1, 'ţ': 1, 'π': 1})


In [60]:
#abcčćdđefghijklmnoprsštuvzž
# Your list of characters
characters = list('abcčćdđefghijklmnoprsštuvzž')

# Create a dictionary to hold character frequencies
char_freq_dict = {char: character_frequency[char] for char in characters}

# Display the character frequencies
print(char_freq_dict)


{'a': 8473320, 'b': 952252, 'c': 512317, 'č': 565108, 'ć': 477359, 'd': 2834497, 'đ': 163320, 'e': 6080196, 'f': 122684, 'g': 1117595, 'h': 519356, 'i': 7050533, 'j': 3419250, 'k': 2461565, 'l': 2239951, 'm': 2543126, 'n': 4648748, 'o': 7134726, 'p': 2027631, 'r': 3511665, 's': 3255595, 'š': 681802, 't': 3651668, 'u': 2828080, 'v': 2702877, 'z': 1281364, 'ž': 410163}


In [61]:
# Calculate the total sum of frequencies
total_sum = sum(char_freq_dict.values())

# Merge counts and percentages into the same dictionary
combined_dict = {char: {'count': freq, 'freq%': round((freq / total_sum) * 100, 3)} for char, freq in char_freq_dict.items()}

# Display the combined dictionary
print(combined_dict)


{'a': {'count': 8473320, 'freq%': 11.823}, 'b': {'count': 952252, 'freq%': 1.329}, 'c': {'count': 512317, 'freq%': 0.715}, 'č': {'count': 565108, 'freq%': 0.789}, 'ć': {'count': 477359, 'freq%': 0.666}, 'd': {'count': 2834497, 'freq%': 3.955}, 'đ': {'count': 163320, 'freq%': 0.228}, 'e': {'count': 6080196, 'freq%': 8.484}, 'f': {'count': 122684, 'freq%': 0.171}, 'g': {'count': 1117595, 'freq%': 1.559}, 'h': {'count': 519356, 'freq%': 0.725}, 'i': {'count': 7050533, 'freq%': 9.838}, 'j': {'count': 3419250, 'freq%': 4.771}, 'k': {'count': 2461565, 'freq%': 3.435}, 'l': {'count': 2239951, 'freq%': 3.126}, 'm': {'count': 2543126, 'freq%': 3.549}, 'n': {'count': 4648748, 'freq%': 6.487}, 'o': {'count': 7134726, 'freq%': 9.955}, 'p': {'count': 2027631, 'freq%': 2.829}, 'r': {'count': 3511665, 'freq%': 4.9}, 's': {'count': 3255595, 'freq%': 4.543}, 'š': {'count': 681802, 'freq%': 0.951}, 't': {'count': 3651668, 'freq%': 5.095}, 'u': {'count': 2828080, 'freq%': 3.946}, 'v': {'count': 2702877, 

In [62]:
# Sort by highest percentage
sorted_combined = dict(sorted(combined_dict.items(), key=lambda item: item[1]['freq%'], reverse=True))

# Display the sorted dictionary by highest percentage
print(sorted_combined)


{'a': {'count': 8473320, 'freq%': 11.823}, 'o': {'count': 7134726, 'freq%': 9.955}, 'i': {'count': 7050533, 'freq%': 9.838}, 'e': {'count': 6080196, 'freq%': 8.484}, 'n': {'count': 4648748, 'freq%': 6.487}, 't': {'count': 3651668, 'freq%': 5.095}, 'r': {'count': 3511665, 'freq%': 4.9}, 'j': {'count': 3419250, 'freq%': 4.771}, 's': {'count': 3255595, 'freq%': 4.543}, 'd': {'count': 2834497, 'freq%': 3.955}, 'u': {'count': 2828080, 'freq%': 3.946}, 'v': {'count': 2702877, 'freq%': 3.771}, 'm': {'count': 2543126, 'freq%': 3.549}, 'k': {'count': 2461565, 'freq%': 3.435}, 'l': {'count': 2239951, 'freq%': 3.126}, 'p': {'count': 2027631, 'freq%': 2.829}, 'z': {'count': 1281364, 'freq%': 1.788}, 'g': {'count': 1117595, 'freq%': 1.559}, 'b': {'count': 952252, 'freq%': 1.329}, 'š': {'count': 681802, 'freq%': 0.951}, 'č': {'count': 565108, 'freq%': 0.789}, 'h': {'count': 519356, 'freq%': 0.725}, 'c': {'count': 512317, 'freq%': 0.715}, 'ć': {'count': 477359, 'freq%': 0.666}, 'ž': {'count': 410163,

In [58]:
import csv

# File path to save the CSV
file_path = 'Frekvencija_glasnika_ParlaSpeechHR_v1.0.csv'

# Writing the sorted dictionary to a CSV file with UTF-8 encoding
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Character', 'Count', 'Freq%']  # Updated field names
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the data rows
    for char, data in sorted_combined.items():
        writer.writerow({'Character': char, 'Count': data['count'], 'Freq%': data['freq%']})

print(f"CSV file '{file_path}' has been saved.")


CSV file 'Frekvencija_glasnika_ParlaSpeechHR_v1.0.csv' has been saved.


In [63]:
import jsonlines

# Path to your JSONL file
jsonl_file_path = 'ParlaSpeech-HR.v1.0.jsonl'

total_word_count = 0

# Read the JSONL file and count words in each line
with jsonlines.open(jsonl_file_path, 'r') as file:
    for line in file:
        if 'words' in line:  # Check if 'words' field exists
            words_list = line['words']  # Assuming 'words' is a list field in each JSON object
            total_word_count += len(words_list)

print(f"Total number of words in the JSONL file: {total_word_count}")


Total number of words in the JSONL file: 14419034


# Frekvencije glasnika u ParlaSpeechHR-v1.0 (v1.0)

Character,Count,Freq%
a,8473320,11.823
o,7134726,9.955
i,7050533,9.838
e,6080196,8.484
n,4648748,6.487
t,3651668,5.095
r,3511665,4.9
j,3419250,4.771
s,3255595,4.543
d,2834497,3.955
u,2828080,3.946
v,2702877,3.771
m,2543126,3.549
k,2461565,3.435
l,2239951,3.126
p,2027631,2.829
z,1281364,1.788
g,1117595,1.559
b,952252,1.329
š,681802,0.951
č,565108,0.789
h,519356,0.725
c,512317,0.715
ć,477359,0.666
ž,410163,0.572
đ,163320,0.228
f,122684,0.171